# Project 2 - Book 1: Ames Housing Data Analysis

## Problem Statement

Our real estate investment company has newly set up a data science team. The main feedback from the invesmtent team is trouble identifying property with investment value due to the myriad of factors. We are tasked to find out what are strong predictors of housing prices. To do so, we will be examining ames housing data. We hope to be able to predict housing prices and therefore identify highly demanded features by potential homeowner so as to provide recommendations to guide the company's investment strategy.

## Executive Summary

We have been tasked to predict housing prices so as to generate actionable insights for the organisation to achieve larger margins in their investment strategy. In order to achieve our goals, we will be performing data cleaning, feature engineering, EDA, feature selection and lastly several regression models to predict sale prices. Based on an accuracy score, the best model will be evaluated and chosen to predict sale prices. Having mirrored the market, we can then find out which are the strong predictors of sale prices. With this information, the company is able to locate properties with the favoured features and flip them for profit, generating value for the management, shareholders and of course customers. 

## Notebooks:
- [Data Cleaning and Feature Selection](./book1_data_cleaning_feature_engineering.ipynb)
- [EDA and Feature Selection](./book2_eda_feature_selection.ipynb)
- [Preprocessing, Modeling and Recommendations](./book3_preprocesing_modeling_recommendations.ipynb)

## Contents:
- [Import Libraries](#Import-Libraries)
- [Import Data](#Import-Data)
- [Data Cleaning](#Data-Cleaning)
- [Feature Engineering](#Feature-Engineering)
- [Save Data to CSV](#Save-Data-to-CSV)

### Import Libraries

In [1]:
import pandas as pd

In [2]:
#display all rows and columns of dataframe
pd.set_option("max_rows", None)
pd.set_option('max_columns', None)

### Import Data

In [3]:
#read data and import into notebook 
ames_train=pd.read_csv('../datasets/train.csv')
ames_test=pd.read_csv('../datasets/test.csv')

In [4]:
#store test data id for reassembling
test_id=[id for id in ames_test.Id]
len(test_id)

879

In [5]:
#check train and test data shapes
print(ames_train.shape)
print(ames_test.shape)
#the difference in number of columns is becausetest data does not contain
#salerprice

(2051, 81)
(879, 80)


#### Concat the two datasets

In [6]:
#concat data for cleaning
ames=pd.concat([ames_train,ames_test],ignore_index=True)
ames.shape

(2930, 81)

In [7]:
ames.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500.0
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000.0
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000.0
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000.0
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500.0


##### First observations:
- Column names contain spacing and are capitalised -> Rename columns
- There are NaN values observed. Check for that
- Check data types

### Data Cleaning

#### Rename Columns

In [8]:
#create a list of names for each column
col_names=[col.lower().replace(' ','_') for col in ames.columns]

#rename columns with list
ames.columns=col_names

#check changes
ames.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500.0
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000.0
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000.0
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000.0
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500.0


#### Check for null values

In [9]:
#check for null values
#because the data set is really big, we first identify the columns that contain null values
null_columns=ames.columns[ames.isnull().any()]
#sum the number of null values in each column in null_columns
ames[null_columns].isnull().sum()

lot_frontage       490
alley             2732
mas_vnr_type        23
mas_vnr_area        23
bsmt_qual           80
bsmt_cond           80
bsmt_exposure       83
bsmtfin_type_1      80
bsmtfin_sf_1         1
bsmtfin_type_2      81
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
electrical           1
bsmt_full_bath       2
bsmt_half_bath       2
fireplace_qu      1422
garage_type        157
garage_yr_blt      159
garage_finish      159
garage_cars          1
garage_area          1
garage_qual        159
garage_cond        159
pool_qc           2917
fence             2358
misc_feature      2824
saleprice          879
dtype: int64

##### Decisions for null values

- For columns with extremely high numbers of nan such as alley, pool and misc_feature, we will be dropping these columns as the lack of data will be statistically insignificant and lead to dimensonality error/bias
- For mas_vnr_xxx/garage_xx/bsmt_xx, it seems like there are missing values because there is no such feature in the property in the first place. We will replace nan with 0 for these variables. 
- For variables such as garage_yr built, we will impute it with the year the house was built which is the norm for most houses with a garage
- Lastly, for continuous variables such as lot frontage, we will be imputing with mean values based on the lot shape and lot area


#### Dropping columns

In [10]:
#these columns have high numbers of missing data -> statistically insignificant
ames.drop(['alley','pool_qc','misc_feature','fence'],axis=1,inplace=True)
ames.shape
#dropped 4 columns

(2930, 77)

#### Imput NaNs with 0

In [11]:
#for most of the columns we will be using for our features, they will likely
#be converted to dummy variable or label encoded. 
# as such, they will likely be 0
#since we will be replacing a large number of columns with na values to 0
#let's build a function to do it
def na_to_0(list_of_col):
    for i in list_of_col:
        ames.fillna({i:0,},inplace=True)

Now let's get to cleaning!

In [12]:
#next we take a look at the columns starting with garage_xx

ames[ames['garage_type'].isnull()].loc[:,['garage_type','garage_yr_blt','garage_finish','garage_qual','garage_cond','garage_area']]

,garage_type,garage_yr_blt,garage_finish,garage_qual,garage_cond,garage_area
28,NaN,NaN,NaN,NaN,NaN,0.0
53,NaN,NaN,NaN,NaN,NaN,0.0
65,NaN,NaN,NaN,NaN,NaN,0.0
79,NaN,NaN,NaN,NaN,NaN,0.0
101,NaN,NaN,NaN,NaN,NaN,0.0
103,NaN,NaN,NaN,NaN,NaN,0.0
114,NaN,NaN,NaN,NaN,NaN,0.0
120,NaN,NaN,NaN,NaN,NaN,0.0
134,NaN,NaN,NaN,NaN,NaN,0.0
136,NaN,NaN,NaN,NaN,NaN,0.0


Here we can see that all the null values in the garage_xxx series are all in the same 113 rows of data
Since we have double checked against garage_area which are zero for the null values (which means there is no garage), we will replace them with 0

For the numerical variables such as garage_yr_blt, we will impute it with yr_built, which we observed to be the same from houses with garages. 

In [13]:
#use our function to replace na with 0
na_to_0(['garage_type','garage_finish','garage_qual','garage_cond'])

In [14]:
#impute garage_yr_blt with yr_built
ames['garage_yr_blt'].fillna(ames['year_built'],inplace=True)

In [15]:
ames[ames['bsmt_exposure'].isnull()].loc[:,['bsmt_qual','bsmt_cond','bsmt_exposure','bsmtfin_type_1','bsmtfin_type_2']].head()
#the bsmt_exposure col has some rows that should be 'No' instead of na
# we will replace those first

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_type_2
12,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN
146,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,NaN,NaN,NaN


In [16]:
#replace na with 'No'
ames.loc[ames['bsmt_cond'] == 'TA', 'bsmt_exposure'] = 'No'

In [17]:
#next, we replace the nan with 0
na_to_0(['bsmt_qual','bsmt_cond','bsmt_exposure','bsmtfin_type_1','bsmtfin_type_2',
         'bsmtfin_sf_1','bsmtfin_sf_2','bsmt_unf_sf','total_bsmt_sf','bsmt_full_bath',
         'bsmt_half_bath'])

In [18]:
#for these 3 columns, we replace nan with 0
na_to_0(['mas_vnr_area','mas_vnr_type','fireplace_qu'])

In [19]:
#since SBrkr is the mode, we will be imputing the NaN in ames['electrical'] with SBrkr
ames['electrical'].fillna('SBrkr',inplace=True)

In [20]:
#lastly, we will take a look at the lot_frontage data
ames['lot_frontage'].unique()
lots=ames[['lot_frontage','lot_area','lot_shape','lot_config']]

#using external research, we have determined that the shape of the lot will give us an accurate 
#estimation of the lot frontage

#firstly, we groupby lot shape to see the mean of lot area and frontage
lot_shapes=lots.groupby('lot_shape').mean()

#next, we calculate a ratio to help determine the missing lot frontage based on
#lot shape and lot area mean
lot_shapes['area_frontage_ratio']=lot_shapes['lot_area']/lot_shapes['lot_frontage']
lot_shapes

,lot_frontage,lot_area,area_frontage_ratio
lot_shape,,,
IR1,74.814985,11544.208376,154.303425
IR2,67.437500,18913.065789,280.453246
IR3,117.636364,32064.187500,272.570373
Reg,66.848871,8865.632598,132.622025


In [21]:
#here we impute the NaN with a mean ratio derived from the lot_area and
#lot shape
area_front_ratio={'IR1':154.12,'IR2':280.45,
                  'IR3':272.57,'Reg':132.49}

ames['lot_frontage']=ames['lot_area']/ames['lot_shape'].map(area_front_ratio)

In [22]:
#check changes
ames['lot_frontage'].head()

0    87.704386
1    74.565274
2    59.793192
3    73.982942
4    92.363094
Name: lot_frontage, dtype: float64

In [23]:
#check for changes
ames.shape

(2930, 77)

### Feature Engineering

Based on some external research, we have identified some factors that are comonnly known to affect the price of a property
- Location
- Property size
- House condition
- Macro Environment

3rd party research: https://resources.point.com/8-biggest-factors-affect-real-estate-prices/


#### Location, Location, Location

In [24]:
#A variable that will affect price is the neighborhood
#since this is an indicator of location
#so we seek to make dummy variables for neighborhood 

neighborhood_dums=pd.get_dummies(ames['neighborhood'],
                                 prefix='nbhd',drop_first=True)
#concat to ames df
ames=pd.concat([ames,neighborhood_dums],axis=1)

In [25]:
#location also related to the accesibility and we have chosen these variables to reflect that
ames['artery_dum']=ames['condition_1'].map(lambda x: 1 if x=='Artery' else 0)
ames['ns_rail_dum']=ames['condition_1'].map(lambda x: 1 if x=='RRAn' else 0)
ames['ew_rail_dum']=ames['condition_1'].map(lambda x: 1 if x=='RRAe' else 0)
ames['feedr']=ames['condition_1'].map(lambda x: 1 if x=='Feedr' else 0)

#### Size does matter

In [26]:
#A variable that pertains to the property size the number of stories
#so we seek to make house_style a dummy variable of two_story
# 1 for 2-stories and 0 for 1 story

#first,we create a dict for the dummies
two_stories_dict={'1Story':0,'1.5Fin':0,'1.5Unf':0,
                  '2Story':1,'2.5Fin':1,'2.5Unf':1,
                  'SFoyer':1, 'SLvl':1}

ames['two_stories']=ames['house_style'].map(two_stories_dict)

#### House Condition

In [27]:
#now we can move on to amenities
#we create a dummy variable for if the house has a garage
ames['garage']=ames['garage_type'].map(lambda x: 0 if x==0 else 1)

In [28]:
#we do label encoding for the kitchen quality
kitchen_qual_dict={'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['kitchen_qual_nom']=ames['kitchen_qual'].map(kitchen_qual_dict)

In [29]:
#we create a dummy variable for if the hosue has central aircon and heating
ames['c_aircon']=ames['central_air'].map(lambda x: 1 if x=='Y' else 0)

heating_dict={'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['heatingqc_nom']=ames['heating_qc'].map(heating_dict)

In [30]:
#we do label encoding for the fireplace quality
fireplace_dict={0:0,'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['fireplace_qu_nom']=ames['fireplace_qu'].map(fireplace_dict)

In [31]:
#we do label encoding for the external quality and condition
bsmt_dict={0:0,'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['bsmt_qual_nom']=ames['bsmt_qual'].map(bsmt_dict)
ames['bsmt_cond_nom']=ames['bsmt_cond'].map(bsmt_dict)

In [32]:
#we do label encoding for the external quality and condition
exter_dict={'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['exter_qual_nom']=ames['exter_qual'].map(exter_dict)
ames['exter_cond_nom']=ames['exter_cond'].map(exter_dict)

In [33]:
#we do label encoding for the garage quality and condition
garage_dict={0:0,'Ex':1,'Gd':2,'TA':3,
                  'Fa':4,'Po':5}

ames['garage_qual_nom']=ames['garage_qual'].map(garage_dict)
ames['garage_cond_nom']=ames['garage_cond'].map(garage_dict)

In [34]:
#lastly, we have the age of the house which is usually an indication of wear and tear
ames['age']=ames['yr_sold']-ames['year_built']

### Macro Environment

In [35]:
yr_sold_dums=pd.get_dummies(ames['yr_sold'],
                                 prefix='yr_sold',drop_first=True)
#concat to ames df
ames=pd.concat([ames,yr_sold_dums],axis=1)

In [36]:
ames.shape

(2930, 125)

### Save Data to CSV

In [37]:
test_clean=ames[ames['saleprice'].isnull()]

In [38]:
test_clean.shape

(879, 125)

In [39]:
test_clean.to_csv('../datasets/test_clean.csv',index=False)

In [40]:
train_clean=ames[~ames['saleprice'].isnull()]

In [41]:
train_clean.shape

(2051, 125)

In [42]:
train_clean.to_csv('../datasets/train_clean.csv',index=False)

In [43]:
ames.shape

(2930, 125)

In [44]:
ames.to_csv('../datasets/combined_clean.csv',index=False)

#### Next Steps:
Having now gotten most of the features that we need to address the issues, we will now proceed to establishing a baseline for our model in the preprocessing and modeling notebook.

- [EDA and Feature Selection](./book2_eda_feature_selection.ipynb)